# Analyse qualitative / exploratoire des motifs

## 1. Chargement du dataset
- Import du fichier `passwords_processed.csv`
- Aperçu initial (head)


- Fréquences de caractères et de classes de caractères
- Substrings / patterns récurrents
- Séquences de chiffres, dates ou motifs particuliers
- Analyse de répétitions ou de similarités

In [1]:
import pandas as pd

df = pd.read_csv("../processed/passwords_processed.csv")
df.head()

,password,label,length_norm,diversity,entropy
0,GkADaiojURMQzIBQ,1,0.80,0.50,0.912070
1,b03p17,0,0.30,0.50,0.310196
2,witwicky,0,0.40,0.25,0.376035
3,914537650,0,0.45,0.25,0.298974
4,m`1^+ax-r72vp1M,1,0.75,1.00,0.983188
